Load data and libraries.

In [1]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/anastasiyashabunevich/Desktop/Kaggle/ashabunevich_key.json"

In [3]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "world_bank_intl_education" dataset
dataset_ref = client.dataset("world_bank_intl_education", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "international_education" table
table_ref = dataset_ref.table("international_education")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the "international_education" table
client.list_rows(table, max_results=10).to_dataframe()

,country_name,country_code,indicator_name,indicator_code,value,year
0,Azerbaijan,AZE,"Population, ages 15-64, female",SP.POP.1564.FE.IN,3.514666e+06,2016
1,"Bahamas, The",BHS,Population of the official entrance age to sec...,UIS.SAP.23.GPV.G1.M,2.474000e+03,2016
2,Barbados,BRB,Population of the official age for secondary e...,SP.SEC.TOTL.FE.IN,9.259000e+03,2016
3,Belarus,BLR,Population of the official age for pre-primary...,SP.PRE.TOTL.FE.IN,1.593500e+05,2016
4,Bermuda,BMU,Official entrance age to upper secondary educa...,UIS.THAGE.3.A.GPV,1.400000e+01,2016
5,Bhutan,BTN,"Lower secondary completion rate, female (%)",SE.SEC.CMPT.LO.FE.ZS,8.523189e+01,2016
6,Nigeria,NGA,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,2.450000e+03,2016
7,Djibouti,DJI,Population of the official age for pre-primary...,SP.PRE.TOTL.IN,3.935700e+04,2016
8,Djibouti,DJI,Population of the official age for primary edu...,SP.PRM.TOTL.IN,9.524600e+04,2016
9,Djibouti,DJI,"Population, ages 15-64, total",SP.POP.1564.TO,6.056380e+05,2016


### 1) Government expenditure on education

Which countries spend the largest fraction of GDP on education?  

To answer this question, consider only the rows in the dataset corresponding to indicator code `SE.XPD.TOTL.GD.ZS`, and write a query that returns the average value in the `value` column for each country in the dataset between the years 2010-2017 (including 2010 and 2017 in the average). 

Requirements:
- Your results should have the country name rather than the country code. You will have one row for each country.
- The aggregate function for average is **AVG()**.  Use the name `avg_ed_spending_pct` for the column created by this aggregation.
- Order the results so the countries that spend the largest fraction of GDP on education show up first.

In [4]:
#query to find countries that spend the largest GDP on education
country_spend_pct_query = """
                          SELECT AVG(value) AS avg_ed_spending_pct, country_name
                          FROM `bigquery-public-data.world_bank_intl_education.international_education`
                          WHERE indicator_code = 'SE.XPD.TOTL.GD.ZS' AND year BETWEEN 2010 AND 2017
                          GROUP BY country_name
                          ORDER BY avg_ed_spending_pct DESC
                          """

# Set up the query to 1 GB
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
country_spend_pct_query_job = client.query(country_spend_pct_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
country_spending_results = country_spend_pct_query_job.to_dataframe()

# View top few rows of results
print(country_spending_results.head())

   avg_ed_spending_pct           country_name
0            12.837270                   Cuba
1            12.467750  Micronesia, Fed. Sts.
2            10.001080        Solomon Islands
3             8.372153                Moldova
4             8.349610                Namibia


### 2) Identify interesting codes to explore

The last question started by telling you to focus on rows with the code `SE.XPD.TOTL.GD.ZS`. But how would you find more interesting indicator codes to explore?

There are 1000s of codes in the dataset, so it would be time consuming to review them all. But many codes are available for only a few countries. When browsing the options for different codes, you might restrict yourself to codes that are reported by many countries.

Write a query below that selects the indicator code and indicator name for all codes with at least 175 rows in the year 2016.

Requirements:
- You should have one row for each indicator code.
- The columns in your results should be called `indicator_code`, `indicator_name`, and `num_rows`.
- Only select codes with 175 or more rows in the raw database (exactly 175 rows would be included).
- To get both the `indicator_code` and `indicator_name` in your resulting DataFrame, you need to include both in your **SELECT** statement (in addition to a **COUNT()** aggregation). This requires you to include both in your **GROUP BY** clause.
- Order from results most frequent to least frequent.

In [5]:
#query selects the indicator code that appears most often during 2016
code_count_query = """
                   SELECT indicator_code, indicator_name, COUNT(1) AS num_rows
                   FROM `bigquery-public-data.world_bank_intl_education.international_education`
                   WHERE year = 2016
                   GROUP BY indicator_name, indicator_code
                   HAVING COUNT(1) >= 175
                   ORDER BY COUNT(1) DESC
                   """

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
code_count_query_job = client.query(code_count_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
code_count_results = code_count_query_job.to_dataframe()

# View top few rows of results
print(code_count_results.head())

      indicator_code                   indicator_name  num_rows
0        SP.POP.GROW     Population growth (annual %)       232
1        SP.POP.TOTL                Population, total       232
2     IT.NET.USER.P2  Internet users (per 100 people)       223
3  SP.POP.1564.FE.IN   Population, ages 15-64, female       213
4     SP.POP.1564.TO    Population, ages 15-64, total       213
